# PRIMERA FUENTE DE DATOS: World Bank

In [14]:
import requests
import pandas as pd
import json

# URL base de la API de World Bank para la anemia infantil
base_url = "http://api.worldbank.org/v2/country/ALL/indicator/SH.ANM.CHLD.ZS"

# Definir los parámetros para la solicitud
params = {
    "format": "json",  # Especificar el formato JSON
}

# Lista para almacenar los datos
all_data = []

# Hacer la solicitud inicial a la API
response = requests.get(base_url, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta a JSON
    
    # Extraer los datos necesarios
    if len(data) > 1:
        all_data = data[1]  # Almacenar los registros de los indicadores
    else:
        print("No se encontraron datos en la respuesta.")
else:
    print("Error al obtener los datos:", response.status_code)

# Convertir los datos extraídos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Ordenar los datos por país y año
df_sorted = df.sort_values(by=["country.value", "date"])

# Guardar los datos en un archivo CSV
df_sorted.to_csv("worldbank_anemia_data_sorted.csv", index=False)

# Guardar también en formato JSON (opcional)
with open("worldbank_anemia_data_sorted.json", "w") as json_file:
    json.dump(all_data, json_file)

# Mostrar las primeras filas del DataFrame
print("Datos extraídos exitosamente:")
print(df_sorted.head(10))  # Mostrar los primeros 10 registros ordenados


Datos extraídos exitosamente:
   countryiso3code  date  value unit obs_status  decimal    indicator.id  \
49             AFE  1974    NaN                        0  SH.ANM.CHLD.ZS   
48             AFE  1975    NaN                        0  SH.ANM.CHLD.ZS   
47             AFE  1976    NaN                        0  SH.ANM.CHLD.ZS   
46             AFE  1977    NaN                        0  SH.ANM.CHLD.ZS   
45             AFE  1978    NaN                        0  SH.ANM.CHLD.ZS   
44             AFE  1979    NaN                        0  SH.ANM.CHLD.ZS   
43             AFE  1980    NaN                        0  SH.ANM.CHLD.ZS   
42             AFE  1981    NaN                        0  SH.ANM.CHLD.ZS   
41             AFE  1982    NaN                        0  SH.ANM.CHLD.ZS   
40             AFE  1983    NaN                        0  SH.ANM.CHLD.ZS   

                                      indicator.value country.id  \
49  Prevalence of anemia among children (% of chil...         ZH 

# SEGUNDA FUENTE: Global Health Observatory

In [13]:
import requests
import pandas as pd
import json

# URL base de la API de nutrición de anemia infantil
base_url = "https://ghoapi.azureedge.net/api/NUTRITION_ANAEMIA_CHILDREN_NUM"

# Definir los parámetros para paginación (si la API la soporta)
params = {
    "$top": 1000,  # Número de registros por página
    "$skip": 0,    # Desplazamiento para la paginación
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()  # Convertir la respuesta a JSON
        all_data.extend(data["value"])  # Agregar los datos a la lista
        
        # Si no hay más datos (menor cantidad de datos que "$top"), se detiene el ciclo
        if len(data["value"]) < params["$top"]:
            break
        
        # Aumentar el desplazamiento para la siguiente página
        params["$skip"] += params["$top"]
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Ordenar los datos por país o región y año
df_sorted = df.sort_values(by=["SpatialDim", "TimeDim"])

# Guardar los datos en un archivo CSV
df_sorted.to_csv("nutrition_anemia_children_sorted.csv", index=False)

# También se puede guardar en formato JSON si lo prefieres
with open("nutrition_anemia_children_sorted.json", "w") as json_file:
    json.dump(all_data, json_file)

print("Datos guardados exitosamente.")

# Mostrar las primeras filas del DataFrame
print(df_sorted.head(10))  # Mostrar los primeros 10 registros ordenados


Datos guardados exitosamente.
            Id                   IndicatorCode SpatialDimType SpatialDim  \
13289  5948686  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
2492   1162290  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
16474  7385086  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
7781   3516992  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
5597   2527270  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
1738    808540  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
17380  7801335  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
16780  7531972  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
7577   3411196  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   
1859    863556  NUTRITION_ANAEMIA_CHILDREN_NUM       UNREGION         11   

      TimeDimType ParentLocationCode ParentLocation Dim1Type  TimeDim  \
13289        YEAR               None           None      SEX

# TERCERA FUENTE: DHS Program

### ENCUESTA 1: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER TIPO DE ANEMIA

In [1]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_ANY"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página
    "page": 1         # Página inicial
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()
        
        # Agregar los datos obtenidos a la lista
        all_data.extend(data.get("Data", []))
        
        # Verificar si se ha llegado al final de los resultados
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar la página para la siguiente solicitud
        params["page"] += 1
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Guardar los datos en un archivo CSV
df.to_csv("dhs_data_anemia_children.csv", index=False)

# También guardar los datos en formato JSON
with open("dhs_data_anemia_children.json", "w") as json_file:
    json.dump(all_data, json_file)

print("Datos guardados exitosamente.")

# Mostrar las primeras 10 filas del DataFrame
print(df.head(10))


Datos guardados exitosamente.
   DataId   SurveyId                 Indicator  IsPreferred  Value  \
0  623614  AL2008DHS  Children with any anemia            1   17.4   
1  273939  AL2017DHS  Children with any anemia            1   24.5   
2  112066  AM2005DHS  Children with any anemia            1   36.5   
3  452445  AM2016DHS  Children with any anemia            1   15.6   
4  170194  AO2011MIS  Children with any anemia            1   53.3   
5  635592  AO2015DHS  Children with any anemia            1   64.8   
6  274255  AZ2006DHS  Children with any anemia            1   39.1   
7  274420  BD2011DHS  Children with any anemia            1   51.3   
8   98735  BF2003DHS  Children with any anemia            1   91.5   
9  274571  BF2010DHS  Children with any anemia            1   87.8   

        SDRID  Precision RegionId SurveyYearLabel SurveyType  ...  \
0  CNANMCCANY          1                  2008-09        DHS  ...   
1  CNANMCCANY          1                  2017-18        DHS 

### ENCUESTA 2: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA LEVE

In [2]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil leve)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MLD"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página
    "page": 1         # Página inicial
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()
        
        # Agregar los datos obtenidos a la lista
        all_data.extend(data.get("Data", []))
        
        # Verificar si se ha llegado al final de los resultados
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar la página para la siguiente solicitud
        params["page"] += 1
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Guardar los datos en un archivo CSV (ajustado para anemia leve)
df.to_csv("dhs_data_anemia_mild_children.csv", index=False)

# También guardar los datos en formato JSON
with open("dhs_data_anemia_mild_children.json", "w") as json_file:
    json.dump(all_data, json_file)

print("Datos de anemia leve guardados exitosamente.")

# Mostrar las primeras 10 filas del DataFrame
print(df.head(10))


Datos de anemia leve guardados exitosamente.
   DataId   SurveyId                  Indicator  IsPreferred  Value  \
0  623613  AL2008DHS  Children with mild anemia            1   11.2   
1  273941  AL2017DHS  Children with mild anemia            1   17.4   
2  112067  AM2005DHS  Children with mild anemia            1   16.5   
3  452447  AM2016DHS  Children with mild anemia            1   11.0   
4  170193  AO2011MIS  Children with mild anemia            1   26.6   
5  635593  AO2015DHS  Children with mild anemia            1   30.7   
6  274252  AZ2006DHS  Children with mild anemia            1   21.3   
7  274423  BD2011DHS  Children with mild anemia            1   29.2   
8   98737  BF2003DHS  Children with mild anemia            1   18.6   
9  274569  BF2010DHS  Children with mild anemia            1   18.2   

        SDRID  Precision RegionId SurveyYearLabel SurveyType  ...  \
0  CNANMCCMLD          1                  2008-09        DHS  ...   
1  CNANMCCMLD          1           

### ENCUESTA 3: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA MODERADA

In [34]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil moderada)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MOD"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página
    "page": 1         # Página inicial
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()
        
        # Agregar los datos obtenidos a la lista
        all_data.extend(data.get("Data", []))
        
        # Verificar si se ha llegado al final de los resultados
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar la página para la siguiente solicitud
        params["page"] += 1
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Guardar los datos en un archivo CSV
df.to_csv("dhs_data_anemia_moderate_children.csv", index=False)

# También guardar los datos en formato JSON
with open("dhs_data_anemia_moderate_children.json", "w") as json_file:
    json.dump(all_data, json_file)

print("Datos guardados exitosamente.")

# Mostrar las primeras 10 filas del DataFrame
print(df.head(10))

Datos guardados exitosamente.
   DataId   SurveyId                      Indicator  IsPreferred  Value  \
0   10271  AL2008DHS  Children with moderate anemia            1    6.1   
1  123088  AL2017DHS  Children with moderate anemia            1    6.6   
2  716693  AM2005DHS  Children with moderate anemia            1   18.7   
3  410697  AM2016DHS  Children with moderate anemia            1    4.2   
4  123503  AO2011MIS  Children with moderate anemia            1   25.5   
5  742510  AO2015DHS  Children with moderate anemia            1   31.8   
6  123570  AZ2006DHS  Children with moderate anemia            1   17.4   
7  123983  BD2011DHS  Children with moderate anemia            1   21.4   
8  664270  BF2003DHS  Children with moderate anemia            1   59.9   
9  124457  BF2010DHS  Children with moderate anemia            1   58.5   

        SDRID  Precision RegionId SurveyYearLabel SurveyType  ...  \
0  CNANMCCMOD          1                  2008-09        DHS  ...   
1  CNA

### ENCUESTA 4: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA GRAVE

In [36]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil moderada)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_SEV"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página
    "page": 1         # Página inicial
}

# Lista para almacenar todos los datos
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos
while True:
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()
        
        # Agregar los datos obtenidos a la lista
        all_data.extend(data.get("Data", []))
        
        # Verificar si se ha llegado al final de los resultados
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar la página para la siguiente solicitud
        params["page"] += 1
    else:
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos a un DataFrame de Pandas
df = pd.json_normalize(all_data)

# Guardar los datos en un archivo CSV
df.to_csv("dhs_data_anemia_moderate_children.csv", index=False)

# También guardar los datos en formato JSON
with open("dhs_data_anemia_moderate_children.json", "w") as json_file:
    json.dump(all_data, json_file)

print("Datos guardados exitosamente.")

# Mostrar las primeras 10 filas del DataFrame
print(df.head(10))


Datos guardados exitosamente.
   DataId   SurveyId                    Indicator  IsPreferred  Value  \
0   10272  AL2008DHS  Children with severe anemia            1    0.1   
1  123091  AL2017DHS  Children with severe anemia            1    0.4   
2  716691  AM2005DHS  Children with severe anemia            1    1.2   
3  410695  AM2016DHS  Children with severe anemia            1    0.4   
4  123504  AO2011MIS  Children with severe anemia            1    1.3   
5  742511  AO2015DHS  Children with severe anemia            1    2.2   
6  123564  AZ2006DHS  Children with severe anemia            1    0.4   
7  123959  BD2011DHS  Children with severe anemia            1    0.7   
8  664269  BF2003DHS  Children with severe anemia            1   13.1   
9  124468  BF2010DHS  Children with severe anemia            1   11.1   

        SDRID  Precision RegionId SurveyYearLabel SurveyType  ...  \
0  CNANMCCSEV          1                  2008-09        DHS  ...   
1  CNANMCCSEV          1    

# CUARTA FUENTE: KAGGLE

In [38]:
pip install kaggle


     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     --------- ---------------------------- 20.5/82.7 kB 131.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/82.7 kB 131.3 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/82.7 kB 272.3 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/82.7 kB 272.3 kB/s eta 0:00:01
     -------------------------------------  81.9/82.7 kB 241.3 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 201.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105795 sha256=15e8cc74f6bc74b090cd18ba4f20b4a5f2628963c1bbb4a949c3604139b395fc
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad

In [39]:
import json
import os

# Ruta al archivo 'kaggle.json'
kaggle_file_path = 'kaggle.json'

# Leer el archivo JSON
with open(kaggle_file_path, 'r') as file:
    kaggle_api = json.load(file)

# Extraer username y key
kaggle_username = kaggle_api['username']
kaggle_key = kaggle_api['key']

# Configurar las variables de entorno para la API de Kaggle
os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_key

print("Credenciales configuradas correctamente.")


Credenciales configuradas correctamente.


In [43]:
import subprocess

# Ejecutar el comando y capturar la salida
result = subprocess.run(
    ["kaggle", "datasets", "list", "-s", "child anemia"],
    capture_output=True,
    text=True
)

# Imprimir la salida
print(result.stdout)  # Muestra el resultado del comando


ref                                                    title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
adeolaadesina/factors-affecting-children-anemia-level  Factors Affecting Children Anemia Level  258KB  2023-10-26 13:13:02           3861         41  1.0              
arashnic/flu-data                                       H1N1 and Seasonal Flu Vaccines          636KB  2020-11-27 18:33:36           1990         31  0.9705882        
ravisinghiitbhu/nfhs5                                  National Family and Health Survey-5        5MB  2022-11-05 08:51:10           1266         32  0.7058824        
jackwin07/celiac-disease-coeliac-disease               Celiac disease (coeliac disease)          10KB  2023-03-24 10:22:16           1014         11  0.47058824

### DATA 1: Factores que afectan el nivel de anemia en los niños (Estudio en Nigeria)



In [50]:
import subprocess

# Nombre del dataset que quieres descargar
dataset_name = "adeolaadesina/factors-affecting-children-anemia-level"

# Descargar el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])

CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'adeolaadesina/factors-affecting-children-anemia-level', '--unzip'], returncode=0)

In [51]:
import pandas as pd

# Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "children anemia.csv"

# Leer el archivo CSV
df = pd.read_csv(csv_file)

# Mostrar las primeras 10 filas
print(df.head(10))



  Age in 5-year groups Type of place of residence Highest educational level  \
0                40-44                      Urban                    Higher   
1                35-39                      Urban                    Higher   
2                25-29                      Urban                    Higher   
3                25-29                      Urban                 Secondary   
4                20-24                      Urban                 Secondary   
5                30-34                      Urban                    Higher   
6                35-39                      Urban                 Secondary   
7                35-39                      Urban                 Secondary   
8                30-34                      Urban                 Secondary   
9                20-24                      Urban                 Secondary   

  Wealth index combined  Births in last five years  \
0               Richest                          1   
1               Riches

### DATA 2: Encuesta Nacional de Familia y Salud (Estudio en ¿India?)

In [52]:
import subprocess

# Nombre del dataset que quieres descargar
dataset_name = "ravisinghiitbhu/nfhs5"

# Descargar el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])

CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'ravisinghiitbhu/nfhs5', '--unzip'], returncode=0)

In [53]:
import pandas as pd

# Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "Final.csv"

# Leer el archivo CSV
df = pd.read_csv(csv_file)

# Mostrar las primeras 10 filas
print(df.head(10))

   Res_Age            State  Edu_level  Water_Source_Time  Toilet_Facility  \
0       38  Jammu & Kashmir          2                  0                1   
1       39  Jammu & Kashmir          1                  0                1   
2       30  Jammu & Kashmir          2                  0                1   
3       32  Jammu & Kashmir          2                  0                1   
4       29  Jammu & Kashmir          2                  0                1   
5       28  Jammu & Kashmir          2                  0                1   
6       47  Jammu & Kashmir          0                 20                1   
7       36  Jammu & Kashmir          2                  0                1   
8       29  Jammu & Kashmir          3                  0                1   
9       36  Jammu & Kashmir          3                  0                1   

   House_electricity  House_radio  House_tv  House_bicycle  House_motorcycle  \
0                  1            1         1              0   